In [1]:
import tensorflow as tf
import importlib
from model.model1D import Unet,Unet_Model
import model.model1D
importlib.reload(model.model1D)
try: 
    del unet
except:
    pass
import numpy as np


import importlib
import dataloader.dataloader
import configs.config
from dataloader.dataloader import Train_Series,TrainSerie,Dataset
importlib.reload(dataloader.dataloader)
importlib.reload(configs.config)
try: 
    del series,ds
except:
    pass



## Load Dataset

In [94]:
import pickle
# Include the epoch in the file name (uses `str.format`)
ds_path = "./model/saved_ds/ds.obj"

with  open(ds_path,'rb') as file:
    ds = pickle.load(file)


## Load Model

In [95]:
# from datetime import datetime

# #save model
# models_path = "model/saved_models/unet_{version}_{now}".format(version = "v1", now = str(datetime.now().date()))
# unet = tf.keras.models.load_model(models_path,compile=False)

# # Check its architecture
# unet.summary()

In [96]:
from model.model1D import Unet,Unet_Model
import model.model1D
importlib.reload(model.model1D)
try: 
    del unet
except:
    pass

unet = Unet(n_classes=3,n_filters=8)
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
unet.model((160,2)).summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 160, 2)]     0           []                               
                                                                                                  
 downsampling_block_35 (Downsam  ((None, 80, 8),     256         ['input_8[0][0]']                
 plingBlock)                     (None, 160, 8))                                                  
                                                                                                  
 downsampling_block_36 (Downsam  ((None, 40, 16),    1184        ['downsampling_block_35[0][0]']  
 plingBlock)                     (None, 80, 16))                                                  
                                                                                            

## Load Weights


In [97]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{epoch:04d}.ckpt"

# Loads the weights
unet.load_weights(checkpoint_path.format(epoch =100))

# Re-evaluate the model
# loss, acc = unet.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


## Load Test Series

In [98]:
from configs.config import CFG
from utils import config
config = config.Config.from_json(CFG)

import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Test_Series
try: 
    del test_series
except:
    pass
test_series = Test_Series(config.data,config.paths)
test_series.createSeries()

In [139]:
def pad_to(x, final_length):
    Ni, h,w = x.shape[:-2], x.shape[-2],x.shape[-1]
    pads = (final_length - h)//2
    shape = (*Ni,final_length,w)
    out = np.zeros(shape=shape)
    if len(Ni) == 0:
        out[pads:(final_length-5)] = x
    else:
        for ii in np.ndindex(Ni):
            x_j = 0
            for jj in range(pads,(final_length-5)):
                out_j = (jj,)
                out[ii+out_j] = x[ii+(x_j,)]
                x_j+=1
        #out[Ni,pads:(final_length-5)] = x
    return out, pads

def unpad(x, pad):
    Ni, h,w = x.shape[:-2], x.shape[-2],x.shape[-1]
    
    if len(Ni) == 0:
        return x[pad:(h-pad)]
    else:
        shape = (*Ni,h-pad*2,w)
        out = np.zeros(shape=shape)
        for ii in np.ndindex(Ni):
            out_j= 0 
            for jj in range(pad,(x.shape[-2]-pad)):
                out[ii+(out_j,)]= x[ii+(jj,)]
                out_j +=1
        return out
        

x = test_series.series["038441c925bb"].get_example()
x = np.expand_dims(x,axis = 0)
x_pad, pads = pad_to(x, 160) # Padded data, feed this to your network 
x_unpad = unpad(x_pad, pads) # Un-pad the network output to recover the original shape

print('Original: ', x.shape)
print('Padded: ', x_pad)
print('Recovered: ', x_unpad)

Original:  (1, 150, 2)
Padded:  [[[ 0.00000e+00  0.00000e+00]
  [ 0.00000e+00  0.00000e+00]
  [ 0.00000e+00  0.00000e+00]
  [ 0.00000e+00  0.00000e+00]
  [ 0.00000e+00  0.00000e+00]
  [ 2.63670e+00  2.17000e-02]
  [ 2.63680e+00  2.15000e-02]
  [ 2.63700e+00  2.16000e-02]
  [ 2.63680e+00  2.13000e-02]
  [ 2.63680e+00  2.15000e-02]
  [ 2.63670e+00  2.17000e-02]
  [ 2.63670e+00  2.17000e-02]
  [ 2.63670e+00  2.18000e-02]
  [ 2.79800e+00  2.23000e-02]
  [ 3.08470e+00  2.17000e-02]
  [ 2.78020e+00  2.29000e-02]
  [ 2.52050e+00  2.16000e-02]
  [ 2.41290e+00  2.18000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.14000e-02]
  [ 2.41290e+00  2.15000e-02]
  [ 2.41320e+00  2.17000e-02]
  [ 4.98744e+01  3.95000e-02]
  [ 5.48498e+01  1.66000e-02]
  [ 4.40993e+01  1.70000e-02]
  [ 6.73813e+01  1.02000e-02]
  [ 6.84605e+01  2.38000e-02]
  [ 3.77219e+01  7.00000e-03]
  [ 3.00025e+01  8.20000e-03]
  [ 2.78679e+01  1.19000e-02]
  [ 2.85856e+01  1.15000e-02]
  [-6.60

## Predict Test Series

In [136]:
def prepare_example(x,ds,final_length):
    x_normpad = ds._transform(x)
    x_normpad = np.expand_dims(x_normpad,axis = 0)
    print(x_normpad.shape)
    x_normpad, pads = pad_to(x_normpad, final_length)
    return x_normpad,pads 


def predict(model,x,pads):
    y_pred = model.predict(x)
    y_pred = unpad(y_pred, pads)  
    events = np.argmax(y_pred,axis = -1,keepdims=True)
    score = np.max(y_pred,axis=-1,keepdims=True)      
    return (events,score)

In [137]:
x = test_series.series["038441c925bb"].get_example()
x,pads = prepare_example(x,ds,160)
x.shape

(1, 150, 2)


(1, 160, 2)

In [138]:

def predict(model,x,pads):
    y_pred = model.predict(x)
    y_pred = unpad(y_pred, pads)  
    events = np.argmax(y_pred,axis = -1,keepdims=True)
    score = np.max(y_pred,axis=-1,keepdims=True)      
    return (events,score)

test_series_pred = {}
for serie_id, test_serie in test_series.series.items():
    x = test_serie.get_example()
    x,pads = prepare_example(x,ds,160)
    events,score = predict(unet,x,pads)
    print(events)

(1, 150, 2)
1/1 [==============================] - 0s 336ms/step
[[[2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]
  [2]]]
(1, 150, 2)
1/1 [================